In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import  precision_recall_curve, roc_auc_score, confusion_matrix, accuracy_score, recall_score, precision_score, f1_score,auc, roc_curve
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.linear_model import Perceptron
from sklearn.neural_network import MLPClassifier

import mlflow
from mlflow.models import infer_signature

import requests


In [3]:
ROOT_PATH = '../data/'
PATH = ROOT_PATH + 'lending_data.csv'
TARGET_COL = 'default.payment.next.month'

SEED = 42

In [4]:
df = pd.read_csv(PATH)

In [5]:
df = df.drop('ID', axis = 1)

In [6]:
train_set, test_set = train_test_split(df, test_size = 0.2, random_state = SEED)

In [7]:
X_train = train_set.drop(['default.payment.next.month'], axis = 'columns')
y_train = train_set['default.payment.next.month']

X_test = test_set.drop(['default.payment.next.month'], axis = 1)
y_test = test_set['default.payment.next.month']

In [8]:
import json

with open('../config/app.json') as f:
    config = json.load(f)

In [9]:
MLFLOW_TRACKING_URI = f"http://localhost:{config['tracking_port']}"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [10]:
model_name = "logistic_reg"
model_version = "champion"
f"models:/{model_name}@{model_version}"


'models:/logistic_reg@champion'

In [11]:
model = mlflow.pyfunc.load_model(f"models:/{model_name}@{model_version}")
model

mlflow.pyfunc.loaded_model:
  artifact_path: lr_pipeline
  flavor: mlflow.sklearn
  run_id: 57289f14dbad401485996fa090eb6ce4

In [12]:
X_test

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6
2308,30000.0,1,2,2,25,0,0,0,0,0,...,11581.0,12580.0,13716.0,14828.0,1500.0,2000.0,1500.0,1500.0,1500.0,2000.0
22404,150000.0,2,1,2,26,0,0,0,0,0,...,116684.0,101581.0,77741.0,77264.0,4486.0,4235.0,3161.0,2647.0,2669.0,2669.0
23397,70000.0,2,3,1,32,0,0,0,0,0,...,68530.0,69753.0,70111.0,70212.0,2431.0,3112.0,3000.0,2438.0,2500.0,2554.0
25058,130000.0,1,3,2,49,0,0,0,0,0,...,16172.0,16898.0,11236.0,6944.0,1610.0,1808.0,7014.0,27.0,7011.0,4408.0
2664,50000.0,2,2,2,36,0,0,0,0,0,...,42361.0,19574.0,20295.0,19439.0,2000.0,1500.0,1000.0,1800.0,0.0,1000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210,360000.0,2,1,2,27,1,-2,-1,-1,-1,...,830.0,0.0,1271.0,179.0,0.0,830.0,0.0,1271.0,179.0,1970.0
14144,500000.0,2,1,2,28,2,0,0,2,0,...,111690.0,78070.0,78376.0,80912.0,6000.0,13151.0,0.0,3000.0,5000.0,10000.0
23108,60000.0,2,1,1,28,0,0,0,0,0,...,48696.0,50385.0,52045.0,52661.0,2100.0,2500.0,2500.0,2500.0,1600.0,1500.0
25703,20000.0,2,1,2,29,1,-1,-1,-1,-1,...,3967.0,1364.0,600.0,0.0,2494.0,3967.0,1370.0,600.0,0.0,0.0


In [13]:
y_preds = model.predict(X_test)
y_preds

array([1, 0, 0, ..., 1, 1, 0], shape=(6000,))

In [14]:
sum(y_preds)

np.int64(2137)

In [15]:
new_df = test_set
new_df['predictions'] = y_preds
new_df

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default.payment.next.month,predictions
2308,30000.0,1,2,2,25,0,0,0,0,0,...,13716.0,14828.0,1500.0,2000.0,1500.0,1500.0,1500.0,2000.0,0,1
22404,150000.0,2,1,2,26,0,0,0,0,0,...,77741.0,77264.0,4486.0,4235.0,3161.0,2647.0,2669.0,2669.0,0,0
23397,70000.0,2,3,1,32,0,0,0,0,0,...,70111.0,70212.0,2431.0,3112.0,3000.0,2438.0,2500.0,2554.0,0,0
25058,130000.0,1,3,2,49,0,0,0,0,0,...,11236.0,6944.0,1610.0,1808.0,7014.0,27.0,7011.0,4408.0,0,1
2664,50000.0,2,2,2,36,0,0,0,0,0,...,20295.0,19439.0,2000.0,1500.0,1000.0,1800.0,0.0,1000.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2210,360000.0,2,1,2,27,1,-2,-1,-1,-1,...,1271.0,179.0,0.0,830.0,0.0,1271.0,179.0,1970.0,0,0
14144,500000.0,2,1,2,28,2,0,0,2,0,...,78376.0,80912.0,6000.0,13151.0,0.0,3000.0,5000.0,10000.0,1,1
23108,60000.0,2,1,1,28,0,0,0,0,0,...,52045.0,52661.0,2100.0,2500.0,2500.0,2500.0,1600.0,1500.0,0,1
25703,20000.0,2,1,2,29,1,-1,-1,-1,-1,...,600.0,0.0,2494.0,3967.0,1370.0,600.0,0.0,0.0,0,1


In [18]:
new_df[(new_df['default.payment.next.month'] == new_df['predictions']) & (new_df['predictions'] == 0)].iloc[10].to_dict()

{'LIMIT_BAL': 200000.0,
 'SEX': 2.0,
 'EDUCATION': 2.0,
 'MARRIAGE': 1.0,
 'AGE': 50.0,
 'PAY_0': -2.0,
 'PAY_2': -2.0,
 'PAY_3': -2.0,
 'PAY_4': -2.0,
 'PAY_5': -2.0,
 'PAY_6': -2.0,
 'BILL_AMT1': 411.0,
 'BILL_AMT2': 453.0,
 'BILL_AMT3': 348.0,
 'BILL_AMT4': 359.0,
 'BILL_AMT5': 672.0,
 'BILL_AMT6': 1620.0,
 'PAY_AMT1': 453.0,
 'PAY_AMT2': 348.0,
 'PAY_AMT3': 359.0,
 'PAY_AMT4': 672.0,
 'PAY_AMT5': 1620.0,
 'PAY_AMT6': 384.0,
 'default.payment.next.month': 0.0,
 'predictions': 0.0}

In [17]:
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 6000 entries, 2308 to 29171
Data columns (total 23 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LIMIT_BAL  6000 non-null   float64
 1   SEX        6000 non-null   int64  
 2   EDUCATION  6000 non-null   int64  
 3   MARRIAGE   6000 non-null   int64  
 4   AGE        6000 non-null   int64  
 5   PAY_0      6000 non-null   int64  
 6   PAY_2      6000 non-null   int64  
 7   PAY_3      6000 non-null   int64  
 8   PAY_4      6000 non-null   int64  
 9   PAY_5      6000 non-null   int64  
 10  PAY_6      6000 non-null   int64  
 11  BILL_AMT1  6000 non-null   float64
 12  BILL_AMT2  6000 non-null   float64
 13  BILL_AMT3  6000 non-null   float64
 14  BILL_AMT4  6000 non-null   float64
 15  BILL_AMT5  6000 non-null   float64
 16  BILL_AMT6  6000 non-null   float64
 17  PAY_AMT1   6000 non-null   float64
 18  PAY_AMT2   6000 non-null   float64
 19  PAY_AMT3   6000 non-null   float64
 20  PAY_AMT4 